In [ ]:
%matplotlib notebook
%precision 3

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

import matplotlib
import matplotlib.pyplot as plt
import datetime
import numpy as np
from matplotlib.animation import FuncAnimation
from datetime import timedelta
import h5py
from matplotlib.pyplot import imshow
import matplotlib.animation as animation
h5py.enable_ipython_completer()
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
import urllib.parse
import urllib.request
import json
import shutil

In [ ]:
# KNMI operational test key from https://developer.dataplatform.knmi.nl/get-started#make-api-calls
key='eyJvcmciOiI1ZTU1NGUxOTI3NGE5NjAwMDEyYTNlYjEiLCJpZCI6IjI4ZWZlOTZkNDk2ZjQ3ZmE5YjMzNWY5NDU3NWQyMzViIiwiaCI6Im11cm11cjEyOCJ9'
     

def getRadarData(key, tstamp):
    url = 'https://api.dataplatform.knmi.nl/open-data/v1/datasets/radar_reflectivity_composites/versions/2.0/files/RAD_NL25_PCP_NA_'+tstamp+'.h5/url'
    headers = {'Authorization': key}

    req = urllib.request.Request(url, headers=headers)
    with urllib.request.urlopen(req) as response:
       meta = response.read()
    
    realurl=json.loads(meta)["temporaryDownloadUrl"]
    req = urllib.request.Request(realurl)
    fname=tstamp+".hf5"
    print(fname)
    with urllib.request.urlopen(req) as response:
        with open(fname, 'wb') as location:
            shutil.copyfileobj(response, location)
            


In [ ]:
now=datetime.datetime.utcnow()
now = now - datetime.timedelta(hours=1, minutes=5)
now -= datetime.timedelta(minutes=now.minute%5)

now.strftime("%Y%m%d%H%M")
start=now
files=[]
for n in range(0,12):
    tstamp=start.strftime("%Y%m%d%H%M")
    files.append(tstamp+".hf5")
    getRadarData(key, tstamp)
    start += datetime.timedelta(minutes=5)

In [ ]:
img = h5py.File(files[0])
list(img) # show what is in there

In [ ]:
# read the first file
img = h5py.File(files[0])
# build the KNMI suggested palette
cmap=np.array(img["visualisation1"]["color_palette"])
knmimap=ListedColormap(cmap/256.0)

# Show the thing
plt.figure()
plt.imshow(img["image1"]["image_data"], cmap=knmimap)

In [ ]:
# Now let's make an animation!

fig, ax = plt.subplots()

def update(fname):
    # clear the axis each frame
    ax.clear()
    ax.set_xlabel("KM")
    ax.set_ylabel("KM")
    ax.grid()
    img = h5py.File(fname)
    cmap=np.array(img["visualisation1"]["color_palette"])
    knmimap=ListedColormap(cmap/256.0)
    
    ax.imshow(img["image1"]["image_data"], cmap=knmimap)
    # replot things
    ax.set_title("KNMI precipitation radar data from "+fname)
    
    
ani = animation.FuncAnimation(fig, update, frames=files, interval=500)
ani.save('dutchrain.gif', writer='imagemagick', fps=2)


In [ ]:
# scan a little deeper in a file
for a in list(img):
    print(a,": ",list(img[a]))

In [ ]:
list(img["visualisation1"]["color_palette"].attrs.items())

In [ ]:
arr = np.ma.fix_invalid(np.array(img["image1"]['image_data']), fill_value=0)
plt.figure()
plt.hist(arr.ravel(), bins=255)